<a href="https://colab.research.google.com/github/marcebalzarelli/Proyecto-Individual-HENRY-N-1-Machine-Learning/blob/main/API%20/%20PI_API_funciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!pip install fastapi uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
import pandas as pd
from pandas import read_csv
import zipfile

app = FastAPI()




In [ ]:
# ABRO DATAFRAME FINAL LIMPIO
ruta_archivo = '/content/drive/MyDrive/df_final_limpio.csv'
df_final= pd.read_csv(ruta_archivo)

In [ ]:
df_final.head(3)

,id,budget,genres,popularity,release_date,revenue,title,vote_average,vote_count,release_year,return,DiaSemana,Mes,cast,crew
0,862,30000000.0,"['Animation', 'Comedy', 'Family']",21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,1995.0,12.451801,Lunes,Octubre,"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...","['John Lasseter', 'Joss Whedon', 'Andrew Stant..."
1,8844,65000000.0,"['Adventure', 'Fantasy', 'Family']",17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,1995.0,4.043035,Viernes,Diciembre,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...","['Larry J. Franco', 'Jonathan Hensleigh', 'Jam..."
2,15602,0.0,"['Romance', 'Comedy']",11.712900,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,1995.0,0.000000,Viernes,Diciembre,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret...","['Howard Deutch', 'Mark Steven Johnson', 'Mark..."


In [ ]:
columnas= df_final.columns
print(columnas)

Index(['id', 'budget', 'genres', 'popularity', 'release_date', 'revenue',
       'title', 'vote_average', 'vote_count', 'release_year', 'return',
       'DiaSemana', 'Mes', 'cast', 'crew'],
      dtype='object')


In [ ]:
from typing import Optional


In [ ]:
@app.get('/cantidad_filmaciones_mes')
def cantidad_filmaciones_mes(Mes: str):
    mes_lower = Mes.lower()   ## para ignorar mayusculas y minusculas
    # Cuento la cantidad de películas estrenadas en el mes
    cantidad = df_final[df_final['Mes'].str.lower() == mes_lower].shape[0]
    return f"{cantidad} cantidad de películas fueron estrenadas en el mes de {Mes}"


@app.get('/cantidad_filmaciones_dia')
def cantidad_filmaciones_dia(Dia: str):
    dia_lower = Dia.lower()  ## para ignorar mayusculas y minusculas
    #Cuento la cantidad de películas estrenadas en el día
    cantidad = df_final[df_final['DiaSemana'].str.lower() == dia_lower].shape[0]
    return f"{cantidad} cantidad de películas fueron estrenadas en los días {Dia}"


@app.get('/score_titulo')
def score_titulo(titulo_de_la_filmación: str):
    titulo_lower = titulo_de_la_filmación.lower() ## para ignorar mayusculas y minusculas
    # Obtengo el título, año de estreno y score
    row = df_final[df_final['title'].str.lower() == titulo_lower].iloc[0]
    titulo = row['title']
    año = row['release_year']
    score = row['popularity']
    return f"La película {titulo} fue estrenada en el año {año} con un score/popularidad de {score}"


@app.get('/votos_titulo')
def votos_titulo(titulo_de_la_filmación: str):
    titulo_lower = titulo_de_la_filmación.lower() ## para ignorar mayusculas y minusculas
    # Obtengo el título, cantidad de votos y valor promedio
    row = df_final[df_final['title'].str.lower() == titulo_lower].iloc[0]
    titulo = row['title']
    año = row['release_year']
    cantidad_votos = row['vote_count']
    promedio_votos = row['vote_average']

    if cantidad_votos < 2000:
        return "La película no cumple la condición de tener al menos 2000 valoraciones."
    else:
        return f"La película {titulo} fue estrenada en el año {año}. La misma cuenta con un total de {cantidad_votos} valoraciones, con un promedio de {promedio_votos}"


@app.get('/get_actor')
def get_actor(nombre_actor: str):
    nombre_actor_lower = nombre_actor.lower()  ## para ignorar mayusculas y minusculas
    # Relleno los valores NaN en las columnas relevantes con valores adecuados para que no me largue error
    df_final['cast'] = df_final['cast'].fillna('')
    df_final['return'] = df_final['return'].fillna(0)
    # Obtengo el éxito del actor, cantidad de filmaciones y promedio de retorno
    actor_data = df_final[df_final['cast'].str.lower().str.contains(nombre_actor_lower, case=False)]
    cantidad_filmaciones = actor_data.shape[0]
    éxito = actor_data['return'].sum()
    promedio_retorno = éxito / cantidad_filmaciones if cantidad_filmaciones > 0 else 0

    return f"El actor {nombre_actor} ha participado de {cantidad_filmaciones} filmaciones. El mismo ha conseguido un retorno de {éxito} con un promedio de {promedio_retorno} por filmación"

@app.get('/get_director')
def get_director(nombre_director: str):
    nombre_director = nombre_director.lower()  ## para ignorar mayusculas y minusculas
    # Relleno los valores NaN en las columnas relevantes con valores adecuados para que no largue error
    df_final['crew'] = df_final['crew'].fillna('')
    df_final['return'] = df_final['return'].fillna(0)
    # Obtengo el éxito del director, detalles de las películas, costo y ganancia
    director_data = df_final[df_final['crew'].str.contains(nombre_director, case=False)]
    éxito = director_data['return'].sum()
    peliculas = []

    for _, row in director_data.iterrows():
        titulo = row['title']
        fecha_lanzamiento = row['release_date']
        retorno_individual = row['return']
        costo = row['budget']
        ganancia = row['revenue']
        peliculas.append({
            'Título': titulo,
            'Fecha de lanzamiento': fecha_lanzamiento,
            'Retorno individual': retorno_individual,
            'Costo': costo,
            'Ganancia': ganancia
        })

    return {
        'Éxito': éxito,
        'Películas': peliculas
    }

In [ ]:
import nest_asyncio
import uvicorn
nest_asyncio.apply()

if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=8000)

INFO:     Started server process [490]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
